title: Change in MCU categories over 6 months
author: Vladas Jankus 
date: 2022-04-04
region: EU  
tags: cards, card, MCU, product, lapse, MCU period, MCU rate, MCU categories
summary: This short research looks into how MCU categories change over a period of 6 months. The main purpose is to see how customers migrate between these categories, and identify which one of them have stable customers in a period 6 months.

In [1]:
cd /app/

/app


In [2]:
import altair as alt
import pandas as pd
from utils.datalib_database import df_from_sql
from IPython.display import HTML, display, Markdown as md

In [3]:
# Create global style for altair charts
def style_theme():
    # Typography
    font = "Karla"
    # At Urban it's the same font for all text but it's good to keep them separate in case you want to change one later.
    labelFont = "Montserrat"
    sourceFont = "Montserrat"
    # Axes
    axisColor = "#000000"
    gridColor = "#DEDDDD"
    # Colors
    markColor = "#88C04D"
    main_palette = [
        "#88C04D",  # green
        "#44A8DA",  # blue
        "#FED234",  # yellow
        "#A989C5",  # purple
        "#066696",
        "#FC97C2",
        "#FF8360",
        "#B7ABC2",
    ]
    sequential_palette = [
        "#cfe8f3",
        "#a2d4ec",
        "#73bfe2",
        "#46abdb",
        "#1696d2",
        "#12719e",
    ]
    return {
        # width and height are configured outside the config dict because they are Chart configurations/properties not chart-elements' configurations/properties.
        "width": 800,  # from the guide
        "height": 400,  # not in the guide
        "config": {
            "title": {
                "fontSize": 18,
                "font": font,
                "anchor": "start",  # equivalent of left-aligned.
                "fontColor": "#000000",
            },
            "axisX": {
                "domain": True,
                "domainColor": axisColor,
                "domainWidth": 1,
                "grid": False,
                "labelFont": labelFont,
                "labelFontSize": 12,
                "labelAngle": 0,
                "tickColor": axisColor,
                "tickSize": 5,  # default, including it just to show you can change it
                "titleFont": font,
                "titleFontSize": 12,
                "titlePadding": 10,  # guessing, not specified in styleguide
                "title": "X Axis Title (units)",
            },
            "axisY": {
                "domain": False,
                "grid": True,
                "gridColor": gridColor,
                "gridWidth": 1,
                "labelFont": labelFont,
                "labelFontSize": 12,
                "labelAngle": 0,
                "ticks": False,  # even if you don't have a "domain" you need to turn these off.
                "titleFont": font,
                "titleFontSize": 12,
                "titlePadding": 10,  # guessing, not specified in styleguide
                "title": "Y Axis Title (units)",
                # titles are by default vertical left of axis so we need to hack this
                "titleAngle": 0,  # horizontal
                "titleY": -15,  # move it up
                "titleX": 18,  # move it to the right so it aligns with the labels
            },
            "range": {
                "category": main_palette,
                "diverging": sequential_palette,
            },
            "legend": {
                "labelFont": labelFont,
                "labelFontSize": 12,
                "symbolType": "square",
                "symbolSize": 100,
                "titleFont": font,
                "titleFontSize": 12,
                "title": "",
                "orient": "none",
                "direction": "horizontal",
                "titleAnchor": "middle",
            },
            "view": {
                "stroke": "transparent",  # altair uses gridlines to box the area where the data is visualized. This takes that off.
            },
            "background": {"color": "#D9E9F0"},  # white rather than transparent
            ### MARKS CONFIGURATIONS ###
            "area": {
                "fill": markColor,
            },
            "line": {
                "color": markColor,
                "stroke": markColor,
                "strokeWidth": 2,
            },
            "trail": {
                "color": markColor,
                "stroke": markColor,
                "strokeWidth": 0,
                "size": 1,
            },
            "path": {
                "stroke": markColor,
                "strokeWidth": 0.5,
            },
            "point": {
                "filled": True,
            },
            "text": {
                "font": sourceFont,
                "color": markColor,
                "fontSize": 11,
                "align": "right",
                "fontWeight": 400,
                "size": 11,
            },
            "bar": {
                "size": 40,
                "binSpacing": 1,
                "opacity": 0.8,
                "continuousBandSize": 30,
                "discreteBandSize": 30,
                "fill": markColor,
                "stroke": False,
            },
            "column": {
                "fontSize": 18,
                "font": font,
                "anchor": "start",  # equivalent of left-aligned.
                "fontColor": "#000000",
            },
        },
    }


alt.themes.register("style_theme", style_theme)
alt.themes.enable("style_theme")

ThemeRegistry.enable('style_theme')

In [4]:
sql_path = "research/product/cards/20220220_mcu_category_change/queries/"

In [5]:
query = open(sql_path + "mcu_groups_august.sql", "r").read()
mcu_groups_august = df_from_sql("redshiftreader", query)

In [6]:
query = open(sql_path + "mcu_groups_change.sql", "r").read()
mcu_groups_change = df_from_sql("redshiftreader", query)

# Change in MCU categories over 6 months
Vladas Jankus<br/>
2022-04-04

In short research we will look at how MCU categories change over a period of 6 months. In an earlier article we have identified several [MCU categores](https://research.tech26.de/reports/20220220_mcu_categories.html#section6), and now we can take a look at how customers change their MCU category over a period of half a year.

### Contents:
* [1. MCU categories absolute change](#section1)
* [2. MCU category change after 6 months](#section2)
* [3. Main insights](#section3)

## TL;DR
Despite smaller findings, there are 4 main things to distinguish:
1. We can notice that continuous MCUs are stable and mostly remain continuously using their card after 6 months. 
2. Sporadic customers really migrate between using their card and being dormant.
3. Distinction between Dormant and Lapsed holds true, as most customers who were lapsed do not return using their card in 6 months. We can confirm that >6months without activity is a good indicator for long term lapse.
4. Undecided customers prove to really be in a transition phase, as nearly all of them end up in a different category after 6 months.

# 1. MCU categories absolute change <a name="section1"></a>

In an [earlier study](https://research.tech26.de/reports/20220220_mcu_categories.html#section6) we have identified 5 different MCU categories. In the first place we want to see how these categories change over time, to understand how customers lapse. Here is a reminder on the categories:

Active MCUs:
1. Continuous MCU - a customer who is using their card continuously over 90% of their time with N26.
2. Sporadic MCU - a customer who keeps lapsing and has over 3 distinct MCU periods
3. Undecided MCU - a customer who has less than 3 distinct MCU periods, but also has not reached 90% of MCU rate.

Lapsed MCUs:
1. Dormant MCU - a customer who is currently inactive, but was active in the past 5 months
2. Lapsed MCU - a customer who is inactive and has been so over 5 months

In the first place we want to look at dynamics of the absolute numbers. In the chart below, we take all our customers that were active 6 months ago and see how their composition changed in terms of MCU categories.

In [7]:
data = mcu_groups_august
# xaxis = 'mcu_category'
xaxis = "on_date"
yaxis = "percentage"

x_axis_title = ""
y_axis_title = ""
chart_title = "MCU categories"

test_header = alt.Header("test")

column_order = ["Continuous", "Sporadic", "Undecided", "Dormant", "Lapsed", "Potential"]

base = alt.Chart(data).encode(
    x=alt.X(xaxis + ":O", axis=alt.Axis(title=x_axis_title), sort=column_order)
)

bar = base.mark_bar().encode(
    y=alt.Y(
        yaxis + ":Q",
        axis=alt.Axis(title=y_axis_title, format=".0%", grid=True),
        scale=alt.Scale(),
    ),
    color=alt.Color("mcu_category", legend=None),
    column=alt.Column("mcu_category:N", title=""),
)
bar_labels = bar.mark_text(dx=40, dy=25, size=40).encode(
    y=alt.Y(yaxis + ":Q", axis=None), text=alt.Text(yaxis + ":N", format=".0%")
)

nearest = alt.selection(
    type="single", nearest=True, on="mouseover", fields=[xaxis], empty="none"
)
selectors = (
    base.mark_point()
    .encode(
        alt.X(xaxis + ":O", sort=column_order),
        opacity=alt.value(0),
    )
    .add_selection(nearest)
)
text_bar = base.mark_text(align="left", dx=-33, dy=25, size=40, color="#fff").encode(
    y=alt.Y(yaxis + ":Q", axis=None),
    text=alt.condition(nearest, yaxis + ":Q", alt.value(" "), format=".0%"),
)
bar_select = bar.mark_bar(size=100).encode(
    y=alt.Y(yaxis + ":Q", axis=None),
    opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
)

bar.properties(
    width=110, height=300, title="MCU categories today vs 6 months ago"
).configure(background="#fff")

alt.Chart(...)

In general we see that the differences in 6 months are not lage. There is a higher rate of lapsed dormant MCUs and less undecided and potential customers. This could possibly be caused by the fact that we have less new customers, so the share of older lapsed customers is larger. We can look at this later, but before that let's see how the customer categories changed over 6 months.

# 2. MCU category change after 6 months <a name="section2"></a>
The chart below displays how the category of MCUs changed over a period of 6 months. *X-axis* displays the category that MCU started with in August 2021, *column colors* show the category where customers ended up in March 2022. You can see that larger sections are labelled for easier recognition. If you want to explore this further, you can look at this [metabase question](https://metabase-product.tech26.de/question/7490) that contains the same chart.

In [8]:
data = mcu_groups_change
xaxis = "start_category"
yaxis = "count"
col_split = "end_category"

x_axis_title = "MCU group 6 months ago"
y_axis_title = "Number of customers"
chart_title = "MCUs split by their MCU rate into activity levels"

column_order = ["Continuous", "Sporadic", "Undecided", "Dormant", "Lapsed", "Potential"]

end_categories = mcu_groups_change["end_category"].unique()
end_cat_color = ["white", "black" "blue", "red", "orange"]

base = alt.Chart(data).encode(
    x=alt.X(xaxis + ":O", axis=alt.Axis(title=x_axis_title), sort=column_order)
)

bar = base.mark_bar(size=100).encode(
    y=alt.Y(
        yaxis + ":Q",
        axis=alt.Axis(title=y_axis_title, grid=True),
        scale=alt.Scale(),
        stack="zero",
    ),
    color=alt.Color(
        col_split, legend=alt.Legend(title="MCU period today", legendX=180, legendY=10)
    ),
)
bar_inside_labels = base.mark_text(size=15, dx=-45, dy=10, align="left").encode(
    y=alt.Y(
        yaxis + ":Q",
        axis=alt.Axis(title=y_axis_title, grid=True),
        scale=alt.Scale(),
        stack="zero",
    ),
    color=alt.Color(col_split, legend=None),
    text=alt.Text("display_text"),
)

nearest = alt.selection(
    type="single", nearest=True, on="mouseover", fields=[xaxis], empty="none"
)
selectors = (
    base.mark_point()
    .encode(
        alt.X(xaxis + ":O", sort=column_order),
        opacity=alt.value(0),
    )
    .add_selection(nearest)
)

bar_select = bar.mark_bar(size=100).encode(
    y=alt.Y(yaxis + ":Q", axis=None, scale=alt.Scale(), stack="zero"),
    opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
)
bar_select_labels = base.mark_text(size=15, dx=-45, dy=-8, align="left").encode(
    y=alt.Y(
        yaxis + ":Q",
        axis=alt.Axis(title=y_axis_title, grid=True),
        scale=alt.Scale(),
        stack="zero",
    ),
    color=alt.Color(
        col_split,
        legend=None,
    ),
    text=alt.Text("display_text"),
    opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
)

alt.layer(
    bar + bar_inside_labels,
    bar_select + bar_select_labels,
    selectors
    # text_bar
).properties(title=chart_title).resolve_scale(y="independent").configure(
    background="#fff"
)

alt.LayerChart(...)

There is really a lot of different information here. Before going into each section, let's focus on the fact that some of these columns mostly maintained the same category, while others gave away most MCUs to other categories. If we look at **continuous, sporadic, lapsed and potential** start categories, at least half of the MCUs are still in the same category after 6 months. Whereas **undecided and dormant** MCUs mostly changed their category after 6 months. It shows that these two categories are more transitional and don't hold MCUs for a long period. 

# 3. Main insights <a name="section3"></a>
Let's summarize the main insights that we can see from the chart above.

1. Continuous MCUs - After 6 months, nearly all continuous MCUs remained in the same category. In general they either remained in the same category or went dormant. It is nice to see that this category is stable, we might want to reduce the ratio of these customers that go dormant and then eventually lapse. Note that we have nearly no lapsed customers here because they need to stay dormant for 6 months to become lapsed.

2. Sporadic and dormant MCUs - There are several interesting insights with sporadic MCUs. In the first place, we can see that they either remained in the same category or became dormant. We could assume that this is normal behaviour for these customers, since they will probably start using their card again. This behaviour can also seen in the dormant column, where a share of customers migrated to the sporadic column. This shows that part of dormant column is the same customer as sporadic. The only difference is the period we capture them. Either on active period, or on their dormant period. Additionally, customers who remained in the dormant column are actually also sporadic, because after 6 months they should have lapsed. If they remained dormant it means that they had a short activity period and are actually sporadic users. This also indicates that MCU categorisation could be slightly amended in the future to account for this.

3. Undecided MCUs - Looking at how undecided MCUs changed over 6 months we can conclude that it's correct to label them as undecided. In 6 months more than half of them changed their category. It shows that these customers just need more time to eventually end up somewhere else.

4. Lapsed MCUs - The chart shows us that the assumption that dormant accounts become lapsed after 6 months of inactivity is true. Nearly all the customers who were lapsed, remained lapsed. It's interesting to note that other customers who were in lapsed column are either dormant or sporadic after 6 months. This could indicate that sometimes sporadic customers can stay dormant for longer than 6 months. However the number is not high enough to change the assumption about lapsed customers.

5. Potential MCUs - Customers who have an account but have never used our card in general don't change after 6 months. This could be an area to explore in the future as they have an open account and might need a push to start using our card.